In [200]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random

# Machine Learning Lib
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation

from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, confusion_matrix

In [1]:
#Load the CSV file as a dataframe
dataset_train = pd.read_csv("messages_train.csv")
dataset_test = pd.read_csv("messages_test.csv")

dataset_train.head()

NameError: name 'pd' is not defined

In [2]:
# Extract the features and the target from the dataframe
x_train = dataset_train.iloc[:, :-1].values
x_test = dataset_test.iloc[:, :-1].values

y_train = dataset_train.iloc[:, -1].values
y_test = dataset_test.iloc[:, -1].values

NameError: name 'dataset_train' is not defined

In [3]:
# Create a column transformer
ct = OneHotEncoder(sparse=False, handle_unknown='ignore')

# Fit and transform the data
x_train = ct.fit_transform(x_train)
x_test = ct.transform(x_test)

NameError: name 'OneHotEncoder' is not defined

In [4]:
# Encode the target
le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

NameError: name 'LabelEncoder' is not defined

In [5]:
# Convert labels to categorical variables
num_classes = len(np.unique(y_train))

y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

NameError: name 'np' is not defined

In [6]:
# Define some variables
max_input = x_train.shape[1]
batch_size = 1
epochs = 5
plot = True

NameError: name 'x_train' is not defined

In [7]:
def create_network(n_dense=1,
                   dense_units=6,
                   activation= 'relu',
                   dropout=Dropout,
                   dropout_rate=0.1,
                   kernel_initializer='glorot_uniform',
                   optimizer='adam',
                   num_classes=1,
                   max_input=x_train.shape[1]):
      # Layer 1
      model = Sequential()
      model.add(Dense(dense_units, input_shape=(max_input,),
                      kernel_initializer=kernel_initializer))
      model.add(Activation(activation))
      model.add(dropout(dropout_rate))

      # Layer 2 to n-1
      for i in range(n_dense - 1):
          model.add(Dense(dense_units, kernel_initializer=kernel_initializer))
          model.add(Activation(activation))
          model.add(dropout(dropout_rate))

      # Layer n
      model.add(Dense(num_classes))
      model.add(Activation('softmax'))
      model.compile(loss='categorical_crossentropy',
                    optimizer=optimizer,
                    metrics=['accuracy'])
      return model

NameError: name 'Dropout' is not defined

In [8]:
# Overide any of the hyperparameters
network = {
    'n_dense': 5,
    'dense_units': 10,
    # 'optimizer': 'Adamax'
}

In [238]:
print("\nModel Complete.")

print('\nBuilding network 1 - Basic IDS...')

model = create_network(num_classes=num_classes, **network)
history_model = model.fit(x_train,
                            y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=1,
                            validation_data=(x_test,y_test),
                            validation_split=0.1)

score_model = model.evaluate(x_test,
                               y_test,
                               batch_size=batch_size,
                               verbose=1)



print('\nNetwork 1 - Basic IDS results')
print('Hyperparameters:', network)
print('Test score:', score_model[0])
print('Test accuracy:', score_model[1])


Model Complete.

Building network 1 - Basic IDS...
Epoch 1/5
1125/1125 [==============================] - 1s 803us/step - loss: 0.6945 - accuracy: 0.5333 - val_loss: 0.6961 - val_accuracy: 0.4262
Epoch 2/5
1125/1125 [==============================] - 1s 687us/step - loss: 0.6768 - accuracy: 0.5956 - val_loss: 0.7204 - val_accuracy: 0.4262
Epoch 3/5
1125/1125 [==============================] - 1s 673us/step - loss: 0.4753 - accuracy: 0.8053 - val_loss: 1.0149 - val_accuracy: 0.4098
Epoch 4/5
1125/1125 [==============================] - 1s 676us/step - loss: 0.2283 - accuracy: 0.9253 - val_loss: 1.3759 - val_accuracy: 0.4590
Epoch 5/5
61/61 [==============================] - 0s 517us/step - loss: 1.5435 - accuracy: 0.5410

Network 1 - Basic IDS results
Hyperparameters: {'n_dense': 5, 'dense_units': 10}
Test score: 1.5434712171554565
Test accuracy: 0.5409836173057556


In [239]:
# Prediction Engine
y_true = np.argmax(y_test, axis=1) 
y_pred = np.argmax(model.predict(x_test), axis=1)
print(y_true, y_pred)

2/2 [==============================] - 0s 3ms/step
[1 1 1 0 0 1 1 0 0 1 1 1 0 0 1 1 0 0 1 0 0 0 0 1 1 1 0 0 1 1 0 1 1 0 0 1 1
 1 1 1 0 0 0 0 1 0 1 1 1 0 1 0 1 1 1 1 1 0 1 0 1] [1 1 1 1 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1]


In [240]:
print("\nCalculating the prediction accuracy ...")
acc = np.sum(y_pred == y_true) / y_test.shape[0]
print("\nTest accuracy of Model: {}%".format(acc * 100))


Calculating the prediction accuracy ...

Test accuracy of Model: 54.09836065573771%


In [241]:
print("Generating the confusion matrix ...")
cm = confusion_matrix(y_true, y_pred)
print(cm)

Generating the confusion matrix ...
[[18  8]
 [20 15]]


In [242]:
print("Generating the classification report ...")
cr = classification_report(y_true, y_pred)
print(cr)

Generating the classification report ...
              precision    recall  f1-score   support

           0       0.47      0.69      0.56        26
           1       0.65      0.43      0.52        35

    accuracy                           0.54        61
   macro avg       0.56      0.56      0.54        61
weighted avg       0.58      0.54      0.54        61



In [243]:
print("\nOutput Classes in Test Dataset\n{}".format(dataset_test['spam'].value_counts()))


Output Classes in Test Dataset
spam
1    35
0    26
Name: count, dtype: int64


In [ ]:
# Fit the model
model.fit(x_train, y_train,
          batch_size=32, # Take 32 at a time
          epochs=5,
          validation_data=(x_test,y_test))

model.evaluate(x=x_test, y=y_test, verbose=0)

In [ ]:
# Create a function to generate the perturbation 
def adversarial_pattern(traffic, label):

  traffic = tf.cast(traffic, tf.float32) # Convert traffic into tensor

  with tf.GradientTape() as tape: # Initating gradients
    tape.watch(traffic)
    prediction = model(traffic)
    loss = tf.keras.losses.MSE(label, prediction)

  gradient = tape.gradient(loss, traffic)
  signed_gradient =tf.sign(gradient)

  return signed_gradient 

In [ ]:
# Create a function to generate the adversarial sample
def generate_adversarials(batch_size):
  while True:
    x=[]
    y=[]
    for batch in range (batch_size):
      N = random.randint(0, 100)

      label = y_train[N]
      traffic = x_train[N]

      perturbations = adversarial_pattern(traffic.reshape(1, x_train.shape[1]), label).numpy() # Creates the noise in the traffic

      epsilon = 0.1  # Reduce the effect of the perturbations (0.1 is the standard)
      adversarial = traffic + perturbations * epsilon

      x.append(adversarial)
      y.append(y_train[N])

    x = np.asarray(x).reshape(batch_size, traffic.shape[0])
    y = np.asarray(y)

    yield x, y

In [ ]:
# Generate 10 adversarial samples
x_test_adv, y_test_adv = next(generate_adversarials(32))

[print('Base accuracy on Adversarial traffic:', model.evaluate(x=x_test_adv, y=y_test_adv, verbose=0), "\n")]

print(x_test_adv[0])

plt.axis('off')
plt.imshow(x_test_adv[0].reshape(2, int(x_train.shape[1]/2)))

In [ ]:
# Original traffic visualization
print("Original Traffic, selected at random")

for i in range(5):
    plt.axis('off')
    plt.imshow(x_train[random.randint(0, len(x_train))].reshape(2, int(x_train.shape[1]/2)))
    plt.show()